In [1]:
import numpy as np
import cv2
import pickle

In [2]:
# Load model Haar-cascade Detection
face_cascade = cv2.CascadeClassifier('../haarcascade_frontalface_default.xml')
#eye_cascade = cv2.CascadeClassifier('D:\Master\Thesis\src\haarcascade_eye.xml')

# video_path = './../../data/video/ch3_26_3_1130_1230.mp4'
video_path = './../../data/video/output1.mp4'
# video_path = './../../data/Target_and_Obstacle_Recognition.mp4'
# load video
cap = cv2.VideoCapture(video_path)

In [3]:
#Calcualte Background

def getForeground(alpha, frame):
    # Set background as first frame
    backGroundModel = frame
    # Update back ground model, apply the background averaging formula:
    # NEW_BACKGROUND = CURRENT_FRAME * ALPHA + OLD_BACKGROUND * (1 - APLHA)
    backGroundModel = frame * alpha + backGroundModel * (1- alpha)
    
    return cv2.absdiff(backGroundModel.astype(np.uint8), frame)

In [4]:
gray_pre = np.zeros((1080,1920),dtype=np.uint8)
pre_face = 0

frame_pos_list = []

while True:
    ret, frame = cap.read()
    print ('frame nunber: %d' % cap.get(1))
    #print (ret)
    
    if ret == True:
        # Show image of video
        #cv2.imshow('input', frame)
        
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        dif = cv2.subtract(gray, gray_pre)
        th, dst = cv2.threshold(dif, 2, 1, cv2.THRESH_BINARY)
        dif_rate = np.sum(dst) / (1080 * 1920)
        gray_pre = gray
        
        if((dif_rate < 0.04) & (pre_face == 0)):
            continue
        
        pre_face = 0
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        
        # Get the foreground
        #foreGround = getForeground(0.01, frame)
        #foreGround = cv2.cvtColor(foreGround, cv2.COLOR_BGR2GRAY)
        
        # Apply thresholding on the background and display the resulting mask
        #ret, mask = cv2.threshold(foreGround, 128, 255, cv2.THRESH_BINARY)
        
#         diff_rate = np.sum(mask)/(1080 * 1920)
#         if(diff_rate < 0.05):
#             continue
            
        print ('aaa')
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        
        if(len(faces) > 0):
            pre_face = 1
            
            frame_idx = cap.get(cv2.CV_CAP_PROP_POS_FRAMES)
            frame_pos_list.append(frame_idx)
            
            name_raw = './../../data/raw_img/' + 'frame%d' % frame_idx + '.png'
            cv2.imwrite(name_raw, frame)
            
            for (x, y, w, h) in faces:
                cv.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
                roi_gray = gray[y:y+h, x:x+w]
                roi_color = frame[y:y+h, x:x+w]
                
                #eyes = eye_cascade.detectMultiScale(roi_gray)
                #for (ex, ey, ew, eh) in eyes:
                #    cv.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
                
            # Save the detected image
            name_detected = './../../data/detected_img/' + 'frame%d' % frame_idx + '.png'
            cv2.imwrite(name_detected, frame)
    else:
        break
    
cap.release()

with open('Frame_detected_list.pkl', 'wb') as f:
    pickle.dump(frame_pos_list, f)
            
        
        
        

frame nunber: 1


error: D:\Build\OpenCV\opencv-3.3.1\modules\objdetect\src\cascadedetect.cpp:1698: error: (-215) !empty() in function cv::CascadeClassifier::detectMultiScale


In [ ]:
### Use Microsoft API to detect face
import requests
from IPython.display import HTML
import cognitive_face as CF
from io import BytesIO
from PIL import Image, ImageDraw

In [ ]:
region = "westcentralus"
subscription_key = "d6ab8eba9c094e9592847e406db1a4fd"
assert subscription_key
face_api_url = 'https://westcentralus.api.cognitive.microsoft.com/face/v1.0/detect'

headers = { 'Ocp-Apim-Subscription-Key': subscription_key }

params = {
    'returnFaceId': 'true',
    'returnFaceLandmarks': 'false',
    'returnFaceAttributes': 'age,gender,headPose,smile,facialHair,glasses,emotion,hair,makeup,occlusion,accessories,blur,exposure,noise',
}